# 人民日报微博抓取示例

### 1. 基础准备（第三方库的安装，标准库与第三方库的引用，目标URL和请求报文头的初始化）

In [ ]:
# beautifulsoup库，lxml库，tqdm库并非python标准库，因此需要自行安装，可使用pip命令快速安装
# 使用pip工具安装beautifulsoup库，lxml库，tqdm库（这里我们引入清华pypi镜像站以提高源码下载速度）
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple beautifulsoup4 lxml tqdm

In [ ]:
# 引入requests库，用于处理发送请求与接收响应
import requests
# 引入beautifulsoup库，用于处理html网页解析
from bs4 import BeautifulSoup
# 引入re库，用于匹配文本
import re
# 引入time包，用于控制请求的频率
import time
# 引入tqdm包，用于监控抓取进度
from tqdm import tqdm

In [ ]:
# 定义抓取人民日报微博帖子的初始URL (%d为整数占位符，可以代表一个整数数字)
begin_url = 'https://weibo.cn/rmrb?page=%d'

# 定义抓取的全部网页URL（抓取N页）
N = 50                          # 设定抓取50页
page_list = []                  # 定义空列表，用于存储全部网页URL
for item in range(1, N + 1):    # 循环遍历1到N+1
    url = begin_url % item      # 拼接单页URL
    page_list.append(url)       # 把单页URL添加到page_list列表内
    
# 构造请求报文头，weibo.cn站点访问的报文请求头需要至少添加账号cookie和用户代理user-agent
# 这两个信息在登陆微博后可以使用chrome浏览器的调试模式F12，NetWork -> Headers -> Requests Headers中找到，复制到对应变量内即可
cookie = '_T_WM=e875e1c9226638113f452f97bde99e47; SCF=AtrTOj7AMx8A58Sqgh7eoC9IoltPjeHcdQG65VL03DQyJJqxIlyL-3g_gALnbwYqqfXHSO48wrfPw6kx8cdX5P8.; SUB=_2A25zl6_XDeRhGeBK6lsW8ijKzT2IHXVRezGfrDV6PUJbkdANLVmhkW1NR_ZNnoEO33Z9EvtsXEC2a1tD2pkMomTa; SUHB=0LcSFl6HDcU8UG; SSOLoginState=1586749319'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
headers = {'cookie': cookie, 'user-agent': user_agent}  # 定义字典，用于存储报文请求头

In [ ]:
# 打印全部网页URL
print(page_list)

In [ ]:
# 打印请求报文头
print(headers)

### 2. 演示单页网页的数据抓取及清洗过程

In [ ]:
# 1. 取出page_list中的第一页
sample_url = page_list[0]       # sample_url = https://weibo.cn/rmrb?page=1
# 打印第一页url
print(sample_url)

In [ ]:
# 2. 使用requests包发送请求，并接收响应
sample_response_body = requests.get(url=sample_url, headers=headers).text
# 打印html网页文本
print(sample_response_body)

In [ ]:
# 3. 使用beautifulsoup包解析html网页文本（使用lxml解析器）
sample_beautifulsoup = BeautifulSoup(sample_response_body, 'lxml')
# 打印解析后的html对象(实际上与上面的文本一致，但是经过beautifulsoup解析后的文本是可以依据标签提取内容的)
print(sample_beautifulsoup)

In [ ]:
# 4. 获取所有包含目标文本的div元素列表
sample_div_list = sample_beautifulsoup.find_all(attrs={"class": "c", "id": re.compile(r'M_I.*?')})
# 打印提取的div块级元素
print(sample_div_list)
# 打印提取的div块级元素数量
print(len(sample_div_list))

In [ ]:
# 5. 遍历div_list，获取目标文本并适当的文本清洗
for sample_div in sample_div_list:
    sample_post = sample_div.text
    sample_clean_post = re.sub(re.compile(r'赞.*?转发.*?评论.*?收藏.*?$'), '', sample_post)  # 清除'赞[]转发[]评论[]收藏...'内容
    sample_clean_post = re.sub(re.compile(r'\[组图共\d张\]'), '', sample_clean_post)        # 清除'组图共n张'内容
    sample_clean_post = sample_clean_post.replace('原图', '').replace('...全文', '')        # 清除'原图'，'...全文'内容
    # 打印文本清洗结果
    print(sample_div_list.index(sample_div)+1, sample_clean_post)

### 3. 完整抓取全部网页的目标数据

In [ ]:
# 正式的循环抓取过程，使用tqdm库监控抓取进度
with open('人民日报微博语料.txt', 'w', encoding='utf-8') as file:
    for url in tqdm(page_list):
        response_body = requests.get(url=url, headers=headers).text
        # 定义每次请求后等待3秒
        time.sleep(3)
        beautifulsoup = BeautifulSoup(response_body, 'lxml')
        div_list = beautifulsoup.find_all(attrs={"class": "c", "id": re.compile(r'M_I.*?')})
        for div in div_list:
            post = div.text
            clean_post = re.sub(re.compile(r'赞.*?转发.*?评论.*?收藏.*?$'), '', post)
            clean_post = re.sub(re.compile(r'\[组图共\d张\]'), '', clean_post)
            clean_post = clean_post.replace('原图', '').replace('...全文', '')
            # 把清洗过的帖子写入上面打开的weibo_post.txt文件内
            file.write('%s\n' % clean_post)
print('抓取完毕')